# Tutorial 4: Analysis of large-scale Stereo-seq MOBS1 data

The following tutorials demonstrates how to use PAST to derive latent embeddings and decipher spatial domains on Stereo-seq mouse olfactory bulb section1 (MOBS1) dataset([Chen, et al., 2022](https://www.sciencedirect.com/science/article/pii/S0092867422003993)). MOBS1 dataset, containing 107,416 spots and 26,145 genes, is too large to analyze with common ST methods. Here we demonstrate PAST could be applied to MOBS1 data with low time and memory cost.

There are two parts in this tutorial:

+ **Integrating self-prior data to analyze MOBS1 data.** This part will show you how to utilize target preprocessed gene expression matrix as self-prior matrix to obtain latent embeddings and spatial clusters on Stereo-seq MOBS1 data.

+ **Integrating external-prior data to analyze MOBS1 data.** This part will show you how to utilize another mouse olfactory bulb section also generated by Stereo-seq to construct external-prior pseudo-bulk matrix for latent feature extraction and spatial clustering on Stereo-seq MOBS1 data.

In [1]:
import past
import os
import scanpy as sc
import warnings
import torch
import numpy as np
import pandas as pd

In [2]:
warnings.filterwarnings("ignore")
sc.set_figure_params(dpi=80, figsize=(3,3), facecolor="white")
os.environ["R_HOME"] = "/home/lizhen/miniconda3/envs/scTECH-R/lib/R"
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

Using memory_profiler to record time and memory cost of PAST for training and prediction

In [3]:
%load_ext memory_profiler

You can uncomment and execute the following command to download the Stereo-seq MOBS1 dataset in AnnData format.

In [4]:
# !wget https://health.tsinghua.edu.cn/software/PAST/data/Stereo-seq_MOBS1.h5ad

In [5]:
## DLPFC DATASET
os.chdir("/home/lizhen/code/PAST/Data")
MOBS1 = sc.read_h5ad(filename="Stereo-seq_MOBS1.h5ad")
MOBS1.var_names_make_unique()
MOBS1

AnnData object with n_obs × n_vars = 107416 × 26145
    obs: 'annotation', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts'
    var: 'n_cells', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'
    obsm: 'spatial'

## Integrating self-prior matrix to analyze MOBS1

We set a random seed for all random process for reproducibility.

In [6]:
past.setup_seed(666)
sdata = MOBS1.copy()

Next, we filter out genes expressed in less than three spots, follow [the Scanpy vignette](https://scanpy-tutorials.readthedocs.io/en/latest/spatial/basic-analysis.html) for spatial transcriptomics to normalize and logarithmize the count matrix and then select the top 3000 spatial variable genes based on the [Geary'C Index](https://www.jstor.org/stable/2986645).

In [ ]:
sdata = past.preprocess(sdata, min_cells=3, is_filter_MT=True, n_tops=3000, gene_method="gearyc")

We initialize PAST model with `d_lat=50`, `k_neighbors=6`.

In [ ]:
PAST_model = past.PAST(d_in=sdata.shape[1], d_lat=50, k_neighbors=6, dropout=0.1).to(device)

Then we train PAST model without specifying external reference data, and PAST will automatically utilize the preprocessed target MOBS1 anndata as self-prior marix.

In [ ]:
%time %memit PAST_model.model_train(sdata, epochs=50, lr=1e-3, batchsize=3600, device=device)

We can obtain the latent embedding through `output()` function of PAST object, and the latent embedding is stored in `sdata.obsm["embedding"]`.

In [ ]:
%time %memit sdata = PAST_model.output(sdata)

The result of mclust and louvain with default parameters is stored in `sdata.obs["mclust"]` and `sdata.obs["Dlouvain"]` respectively.

In [ ]:
sdata = past.mclust_R(sdata, num_cluster=sdata.obs["annotation"].nunique(), used_obsm='embedding')
sdata = past.default_louvain(sdata, use_rep="embedding")

Evaluation of latent embedding and spatial clustering results.

In [ ]:
print("Cross-validation score:", end="\t\t")
acc, kappa, mf1, wf1 = past.svm_cross_validation(sdata.obsm["embedding"], sdata.obs["annotation"])
print("Acc: %.3f, K: %.3f, mF1: %.3f, wF1: %.3f"%(acc.mean(), kappa.mean(), mf1.mean(), wf1.mean()))

print("Mclust metrics:", end="\t\t\t")
ari, ami, nmi, fmi, comp, homo = past.cluster_metrics(sdata, "annotation", "mclust")
print("ARI: %.3f, AMI: %.3f, NMI: %.3f, FMI:%.3f, Comp: %.3f, Homo: %.3f"%(ari, ami, nmi, fmi, comp, homo))

print("Dlouvain metrics:", end="\t\t")
ari, ami, nmi, fmi, comp, homo = past.cluster_metrics(sdata, "annotation", "Dlouvain")
print("ARI: %.3f, AMI: %.3f, NMI: %.3f, FMI:%.3f, Comp: %.3f, Homo: %.3f"%(ari, ami, nmi, fmi, comp, homo))

Visualization of latent embeddings colored by annotation and PAST-derived clusters.

In [ ]:
sc.pp.neighbors(sdata, use_rep='embedding')
sc.tl.umap(sdata)
sc.pl.umap(sdata, color=["annotation", "mclust"], title=["Ground Truth", "PAST-S"])

Visualization of spatial domains colored by annotation and PAST-derived clusters.

In [ ]:
sc.pl.embedding(sdata, color=["annotation", "mclust"], basis="spatial", title=["Ground Truth", "PAST-S"])

## Integrating external-prior matrix to analyze MOBS1

Similarly, We set a random seed for all random process for reproducibility.

In [ ]:
past.setup_seed(666)
sdata = MOBS1.copy()

As done above, We also filter out genes expressed in less than three spots, follow [the Scanpy vignette](https://scanpy-tutorials.readthedocs.io/en/latest/spatial/basic-analysis.html) for spatial transcriptomics to normalize and logarithmize the count matrix and then select the top 3000 spatial variable genes based on the [Geary'C Index](https://www.jstor.org/stable/2986645).

In [ ]:
sdata = past.preprocess(sdata, min_cells=3, is_filter_MT=True, n_tops=3000, gene_method="gearyc")

Here we utilize another mouse olfactory bulb section2 (MOBS2) dataset([Chen, et al., 2022](https://www.sciencedirect.com/science/article/pii/S0092867422003993)) as reference data, denoted as `rdata`.

In [ ]:
os.chdir("/home/lizhen/code/PAST/Data")
rdata = sc.read_h5ad(filename="Stereo-seq_MOBS2.h5ad")
rdata = past.integration(rdata, sdata)
rdata = past.preprocess(rdata, min_cells=None, target_sum=None, is_filter_MT=False, n_tops=None)
rdata = past.get_bulk(rdata, key="annotation")
rdata

We initialize PAST model with the same parameters.

In [ ]:
PAST_model = past.PAST(d_in=sdata.shape[1], d_lat=50, k_neighbors=6, dropout=0.1).to(device)

Then we use `rdata` to construct external-prior matrix, and `rdata` should contain raw count gene expression matrix of tissues related to the target data. Besides, users need to specify the `anno_key` stored in `rdata.obs` because the construction of external-prior pseudo-bulk matrix also requires fine annotation of each spot.

In [ ]:
%time %memit PAST_model.model_train(sdata, rdata=rdata, epochs=50, lr=1e-3, batchsize=3600, device=device)

We then obtain the latent embedding through `output()` function of PAST object, and the latent embedding is stored in `sdata.obsm["embedding"]`.

In [ ]:
%time %memit sdata = PAST_model.output(sdata)

The result of mclust and louvain with default parameters is stored in `sdata.obs["mclust"]` and `sdata.obs["Dlouvain"]` respectively.

In [ ]:
sdata = past.mclust_R(sdata, num_cluster=sdata.obs["annotation"].nunique(), used_obsm='embedding')
sdata = past.default_louvain(sdata, use_rep="embedding")

Evaluation of latent embedding and spatial clustering results.

In [ ]:
print("Cross-validation score:", end="\t\t")
acc, kappa, mf1, wf1 = past.svm_cross_validation(sdata.obsm["embedding"], sdata.obs["annotation"])
print("Acc: %.3f, K: %.3f, mF1: %.3f, wF1: %.3f"%(acc.mean(), kappa.mean(), mf1.mean(), wf1.mean()))

print("Mclust metrics:", end="\t\t\t")
ari, ami, nmi, fmi, comp, homo = past.cluster_metrics(sdata, "annotation", "mclust")
print("ARI: %.3f, AMI: %.3f, NMI: %.3f, FMI:%.3f, Comp: %.3f, Homo: %.3f"%(ari, ami, nmi, fmi, comp, homo))

print("Dlouvain metrics:", end="\t\t")
ari, ami, nmi, fmi, comp, homo = past.cluster_metrics(sdata, "annotation", "Dlouvain")
print("ARI: %.3f, AMI: %.3f, NMI: %.3f, FMI:%.3f, Comp: %.3f, Homo: %.3f"%(ari, ami, nmi, fmi, comp, homo))

Visualization of latent embeddings colored by annotation and PAST-derived clusters.

In [ ]:
sc.pp.neighbors(sdata, use_rep='embedding')
sc.tl.umap(sdata)
sc.pl.umap(sdata, color=["annotation", "mclust"], title=["Ground Truth", "PAST-E"])

Visualization of spatial domains colored by annotation and PAST-derived clusters.

In [ ]:
sc.pl.embedding(sdata, color=["annotation", "mclust"], basis="spatial", title=["Ground Truth", "PAST-E"])